In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import level2_roi_extractor
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

In [3]:
# #filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
# sst_roi_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|finger|motor|response inhibition')]

# ## SST
# train_betas_with_data = get_data_for_confirmed_train_subjs(
#     beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
#     nonbids_data_path = config['nonbids_data_path'],
#     #ml_data_folderpath = ml_data_folderpath,
#     ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
#     dropbox_datapath=config['dropbox_data_dir'],
#     exclude_test_subjs=False
# )
# train_betas_with_data['wave']=1

# #we're not interestd in getting contrasts; comment this out.
# #betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
# betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])


### W1 W2 analysis

In [4]:
#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/',
    config['mask_location'] + 'striatum/',
    config['mask_location']
]
wave_compare_roi_df = get_mask_df_from_mask_locations(mask_locations)

In [5]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = wave_compare_roi_df.loc[wave_compare_roi_df['mask_label'].str.contains('response inhibition|striatum_joint_reward|value')]


In [28]:
motion_exclusions = pd.read_csv(config['dropbox_data_dir'] + '/DEVQC_all_subjects - All.csv', header=1)

In [29]:
motion_exclusions

,subjectID,wave,Exclude,SST_Exclude,ROC_Exclude,WTP_Exclude,ends in,Coder,notes,anatomical,...,WTP1 motion,WTP2 SDC,WTP2 reg,WTP2 motion,WTP3 SDC,WTP3 reg,WTP3 motion,WTP4 SDC,WTP4 reg,WTP4 motion
0,DEV001,1,NaN,NaN,NaN,NaN,1,BJS,NaN,NaN,...,0,x,x,0,x,x,0,x,x,0
1,DEV001,2,NaN,NaN,NaN,NaN,1,BJS,NaN,NaN,...,0,missing,x,0,missing,x,1,missing,x,NaN
2,DEV004,1,NaN,NaN,NaN,NaN,4,DFA,NaN,NaN,...,0,x,x,0,x,x,0,x,x,0
3,DEV004,2,NaN,NaN,NaN,NaN,4,DFA,NaN,NaN,...,0,x,x,0,x,x,0,x,x,NaN
4,DEV005,1,NaN,NaN,NaN,NaN,5,BJS,NaN,NaN,...,0,x,x,0,x,x,0,x,x,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,DEV311,2,Exclude (PFC Only),Exclude (PFC Only),Exclude (PFC Only),Exclude (PFC Only),1,BJS,NaN,NaN,...,x,x,x,x,x,x,x,x,x,x
567,DEV312,1,NaN,NaN,NaN,NaN,2,BJS,NaN,NaN,...,x,x,x,x,x,x,x,x,x,x
568,DEV312,2,NaN,NaN,NaN,NaN,2,BJS,NaN,NaN,...,x,x,x,x,x,x,x,x,x,x
569,DEV999,1,NaN,NaN,NaN,NaN,9,SW,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:

## SST
train_betas_with_data = get_data_for_confirmed_task_session(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    subj_wave_inclusion='all'
)

train_betas_with_data['wave']='all'

loaded 275 rows from data_by_ppt.csv
479 sessions with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
7 subjects excludeable for missing scan data.
470 sessions remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
470 sessions remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 250


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_44411/3158860866.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_betas_with_data.sort_values('subject_id', inplace=True)
/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_44411/374063233.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_betas_with_data['wave']='all'


In [23]:
len(train_betas_with_data)

159

In [7]:
contrasts_with_paths = get_contrasts_for_betas(train_betas_with_data)

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV008/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_cond

### Spatial normalization

In [ ]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)'
    ]

l2_roi_extractor = level2_roi_extractor(center_data=False,scale_data=False)
l2_roi_extractor.load_all_images_simultaneously=True
l2_roi_extractor.image_standardize = True

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)

In [15]:
roi_data_sst_conditions.to_csv(config['dropbox_data_dir'] + '/subject_sst_health_avg_roi_data_raw_zscored.csv')

See this doc to demonstrate why we need to normalize across the image, not normalize across the subjects:

https://docs.google.com/spreadsheets/d/15iykB7qzCm8xqt7hgzIH58e-pAWgGz9BWKjEeN0MpBw/edit#gid=0


# TEMP